In [7]:
# Write a Python script to download IMDB 250 Top Rated Movies.
# For each movie, you'll need to retrieve the movie title, the ranking, the initial release year, the casting and
# the rating.
# Your data must be stored in a proper imdb_top_250.csv file.

import requests, csv, re
from bs4 import BeautifulSoup
import pandas as pd

def get_movies(url):
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.6167.186 Safari/537.36"
    headers = {"User-Agent": user_agent}
    
    response = requests.get(url, headers=headers)
    soup_imdb = BeautifulSoup(response.text, 'html.parser')
    
    all_movies = soup_imdb.find_all("li", {"class":"ipc-metadata-list-summary-item"})

    # would have been more efficient to put all of these in 1 for loop
    # that way it isnt iterating through all_movies 4 times
    # im sure theres a better way is what im saying, but this is the format i thought you wanted
    
    # get titles :
    # i HATE pythons implementation of regex  
    # movie title is found in 'h3' tags within the movie div, so thats all i need to search for here
    titles = [re.search(r" (.*)", movie.find('h3').contents[0]).group()[1:] for movie in all_movies]

    # get ranking :
    # can use the same 'h3' tag since the ranking occures before the movie title
    rankings = [re.search(r"\d*", movie.find('h3').contents[0]).group() for movie in all_movies]
    
    # get release year :
    years = [int(movie.find('span', 'sc-b0691f29-8 ilsLEX cli-title-metadata-item').contents[0]) for movie in all_movies]

    # get rating :
    # the rating was found in the aria-label attribute of a span. it was also found as a #text but grabbing it from
    # the aria-label was easy enough
    ratings = [float(re.search(r'\d\.\d', movie.find('span', 'ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating').attrs['aria-label']).group()) for movie in all_movies]


    # preparation of data
    d = {"ranking": rankings, "title": titles, "year": years, "rating": ratings}
    df = pd.DataFrame(data=d)
    print(df)

    # write to csv
    df.to_csv('movies.csv')

    # i didnt need the below, but it couldve been used in place of a dataframe

    #mega_list = list(zip(titles, rankings, years, ratings))
    #mega_list = [list(elt) for elt in mega_list]

    # Write csv
    #str_lst = [f"{mega_list[i][1]} / {mega_list[i][0]} ({mega_list[i][2]}) / Starring: {mega_list[i][3]}" \
    #for i, val in enumerate(mega_list)]

    #res = [elt.split("/") for elt in str_lst]

    #with open("imdb_top_250.csv", "w") as f:
        #writer = csv.writer(f, delimiter="-")
        #writer.writerows(res)
def main():
    url = "https://www.imdb.com/chart/top"
    get_movies(url)


if __name__ == "__main__":
    main()

    ranking                     title  year  rating
0         1  The Shawshank Redemption  1994     9.3
1         2             The Godfather  1972     9.2
2         3           The Dark Knight  2008     9.0
3         4    The Godfather: Part II  1974     9.0
4         5              12 Angry Men  1957     9.0
..      ...                       ...   ...     ...
245     246                  The Help  2011     8.1
246     247     It Happened One Night  1934     8.1
247     248        Dances with Wolves  1990     8.0
248     249                   Aladdin  1992     8.0
249     250                  Drishyam  2015     8.2

[250 rows x 4 columns]
